# Experiment 2 - Ablation Study

### Imports

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.linear_model import LinearRegression

In [2]:
### Extract exp3 parameters (betas) from the df_total.gz file, obtained from LCDB

pd.set_option('use_inf_as_na',True)
df_total = pd.read_pickle('data/df_total.gz').dropna()

data = df_total[(df_total['curve_model'] == 'exp3')]
data = data.loc[data.groupby(['openmlid', 'learner'])['max_anchor_seen'].idxmax()]

q50 = data['MSE_tst_last'].quantile(0.5)
data = data[data['MSE_tst_last'] < q50]

x = [x[0] for x in data['beta']]
y = [x[1] for x in data['beta']]
z = [x[2] for x in data['beta']]

betas = pd.DataFrame(zip(x,y,z), columns=['a', 'b', 'c'])

### Metric

In [5]:
def gen_params(idx):
    a = betas['a'].iloc[idx]
    b = betas['b'].iloc[idx]
    c = betas['c'].iloc[idx]
    
    return a,b,c


def run_experiment():
    n = int(len(betas)) # number of LCs considered
    true_pos = 0
    true_neg = 0
    false_pos = 0
    false_neg = 0
    asc = 0
    desc = 0
    
    for i in tqdm(range(0,n)):
        is_asc = False
        flag = False
        final_slopes = []
        
        lern = data['learner'].iloc[i]
        dataset = data['openmlid'].iloc[i]
        
        a,b,c = gen_params(i)

        exp3 = lambda x: a * np.exp((-b) * x) + c # exp3 from LCDB
        anch = data['anchor_prediction'].iloc[i] # array of anchors
        
        if (a < 0 and b > 0) or (a > 0 and b < 0):
            is_asc = True
            asc+=1
        else:
            is_asc = False
            desc+=1
        
        for j in range(0, len(anch)):
            points = range(anch[j] - 10, anch[j] + 11)
            errors_iterations = []
            slopes = []
            for it in range(0, 25):
                noise = np.random.normal(0,0.002)
                linreg_errs = [exp3(p)+noise for p in points]
        
                model = LinearRegression()
                model.fit(np.array(points).reshape(-1,1), np.array(linreg_errs).reshape(-1,1))
                
                if(model.coef_[0] > 0):
                    slopes.append(1)
                else:
                    slopes.append(-1)
            final_slopes.append(np.mean(slopes))
         
        for j in range(0, len(anch)):
            if final_slopes[j] > 0:
                if is_asc:
                    true_pos+=1
                else:
                    false_pos+=1
                flag = True
                break
                
        if flag == False:
            if is_asc:
                false_neg+=1
            else:
                true_neg+=1


    print(f"Out of {n} Learning curves:")
    print(f"-----------{asc} Non-monotonic LCs")
    print(f"-----------{desc} Monotonic LCs\n")
    print(f" - {true_pos} Have been classified correctly as non-monotonic ({round((true_pos/asc)*100,2)}%)")
    print(f" - {true_neg} Have been classified correctly as monotonic ({round((true_neg/desc)*100,2)}%)")
    print(f" - {false_pos} Have been classified incorrectly as non-monotonic (Type I Error)")
    print(f" - {false_neg} Have been classified incorrectly as monotonic (Type II Error)")
    return

In [6]:
run_experiment()

100%|██████████████████████████████████████████████████████████████████████████████| 3769/3769 [09:39<00:00,  6.50it/s]

Out of 3769 Learning curves:
-----------3179 Non-monotonic LCs
-----------590 Monotonic LCs

 - 3167 Have been classified correctly as non-monotonic (99.62%)
 - 355 Have been classified correctly as monotonic (60.17%)
 - 235 Have been classified incorrectly as non-monotonic (Type I Error)
 - 12 Have been classified incorrectly as monotonic (Type II Error)


In [7]:
monotonic_lcs = [] # contains indexes from data DataFrame of the monotonic LCs.

for i in range(0, len(betas)):
    a,b,c = gen_params(i)
    if (a > 0 and b > 0) or (a < 0 and b < 0):
        monotonic_lcs.append(i)    

In [25]:
def peaking_experiment():
    n = len(monotonic_lcs)
    
    corr_nonmon_abl = 0 # correctly classified as non-monotonic when 1 anchor point is considered (ablation)
    corr_nonmon_norm = 0 # correctly classified as non-monotonic when 2 anchor point are considered (initial metric)
    
    for i in tqdm(range(0, n)):
        final_slopes = []
        idx = monotonic_lcs[i]
        a,b,c = gen_params(idx)
        lern = data['learner'].iloc[idx]
        dataset = data['openmlid'].iloc[idx]
        
        exp3 = lambda x: a * np.exp((-b) * x) + c # exp3 from LCDB
        anch = data['anchor_prediction'].iloc[idx] # array of anchors
        
        for j in range(0, len(anch)):
            points = range(anch[j] - 10, anch[j] + 11)
            errors_iterations = []
            slopes = []
            for it in range(0, 25):
                noise = np.random.normal(0,0.002)
                linreg_errs = None
                if j == 5:
                    linreg_errs = [exp3(j-1)+abs(noise) for p in points]
                else:
                    linreg_errs = [exp3(p)+noise for p in points]
                
                model = LinearRegression()
                model.fit(np.array(points).reshape(-1,1), np.array(linreg_errs).reshape(-1,1))
                
                if(model.coef_[0] > 0):
                    slopes.append(1)
                else:
                    slopes.append(-1)
            final_slopes.append(np.mean(slopes))
         
        for j in range(0, len(anch)):
            if final_slopes[j] > 0:
                corr_nonmon_abl += 1
                if j < len(anch) - 1 and final_slopes[j+1] > 0:
                    corr_nonmon_norm += 1
                break
            
    print(f"Out of {n} monotonic LCs with peaking at anchor 5:")
    print(f" - {corr_nonmon_abl} have been correctly classified as non-monotonic on ablation ({round((corr_nonmon_abl/n)*100,2)}%)")
    print(f" - {corr_nonmon_norm} have been correctly classified as non-monotonic on normal metric ({round((corr_nonmon_norm/n)*100,2)}%)")

In [26]:
peaking_experiment()

100%|████████████████████████████████████████████████████████████████████████████████| 590/590 [02:02<00:00,  4.81it/s]

Out of 590 monotonic LCs with peaking at anchor 5:
 - 250 have been correctly classified as non-monotonic on ablation (42.37%)
 - 22 have been correctly classified as non-monotonic on normal metric (3.73%)
